## Importing libraries

In [1]:
import pypots
import os
import sys
from pypots.utils.metrics import calc_mae
from pypots.optim import Adam
from pypots.imputation import SAITS, BRITS, USGAN, GPVAE, MRNN
import numpy as np
import benchpots
from pypots.utils.random import set_random_seed
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import pandas as pd    
from pypotsModify.benchpotsMAE.datasets import preprocess_physionet2012

2025-02-10 22:28:50.520411: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-10 22:28:50.536362: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-10 22:28:50.541243: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-10 22:28:50.553327: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.



████████╗██╗███╗   ███╗███████╗    ███████╗███████╗██████╗ ██╗███████╗███████╗    █████╗ ██╗
╚══██╔══╝██║████╗ ████║██╔════╝    ██╔════╝██╔════╝██╔══██╗██║██╔════╝██╔════╝   ██╔══██╗██║
   ██║   ██║██╔████╔██║█████╗█████╗███████╗█████╗  ██████╔╝██║█████╗  ███████╗   ███████║██║
   ██║   ██║██║╚██╔╝██║██╔══╝╚════╝╚════██║██╔══╝  ██╔══██╗██║██╔══╝  ╚════██║   ██╔══██║██║
   ██║   ██║██║ ╚═╝ ██║███████╗    ███████║███████╗██║  ██║██║███████╗███████║██╗██║  ██║██║
   ╚═╝   ╚═╝╚═╝     ╚═╝╚══════╝    ╚══════╝╚══════╝╚═╝  ╚═╝╚═╝╚══════╝╚══════╝╚═╝╚═╝  ╚═╝╚═╝
ai4ts v0.0.3 - building AI for unified time-series analysis, https://time-series.ai 



/usr/local/lib/python3.11/dist-packages/pypots/nn/modules/reformer/local_attention.py:31: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled=False)
/usr/local/lib/python3.11/dist-packages/pypots/nn/modules/reformer/local_attention.py:98: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled=False)


## Loading database

#### StandardScaler

In [2]:
set_random_seed()

# Load the PhysioNet-2012 dataset
physionet2012_dataset = preprocess_physionet2012(subset="all", rate=0.1, normalization=1)

# Take a look at the generated PhysioNet-2012 dataset, you'll find that everything has been prepared for you,
# data splitting, normalization, additional artificially-missing values for evaluation, etc.
print(physionet2012_dataset.keys())

2025-02-10 22:28:58 [INFO]: Have set the random seed as 2022 for numpy and pytorch.
2025-02-10 22:28:58 [INFO]: You're using dataset physionet_2012, please cite it properly in your work. You can find its reference information at the below link: 
https://github.com/WenjieDu/TSDB/tree/main/dataset_profiles/physionet_2012
2025-02-10 22:28:58 [INFO]: Dataset physionet_2012 has already been downloaded. Processing directly...
2025-02-10 22:28:58 [INFO]: Dataset physionet_2012 has already been cached. Loading from cache directly...
2025-02-10 22:28:58 [INFO]: Loaded successfully!
2025-02-10 22:29:12 [WARNING]: Note that physionet_2012 has sparse observations in the time series, hence we don't add additional missing values to the training dataset. 
2025-02-10 22:29:12 [WARNING]: Note that physionet_2012 has sparse observations in the time series, hence we don't add additional missing values to the training dataset. 
2025-02-10 22:29:12 [INFO]: 68807 values masked out in the val set as ground t

dict_keys(['n_classes', 'n_steps', 'n_features', 'scaler', 'train_X', 'train_y', 'train_ICUType', 'val_X', 'val_y', 'val_ICUType', 'test_X', 'test_y', 'test_ICUType', 'female_gender_test_X', 'female_gender_test_y', 'test_ICUType_female_gender', 'male_gender_test_X', 'male_gender_test_y', 'test_ICUType_male_gender', 'undefined_gender_test_X', 'undefined_gender_test_y', 'test_ICUType_undefined_gender', 'more_than_or_equal_to_65_test_X', 'more_than_or_equal_to_65_test_y', 'test_ICUType_more_than_or_equal_to_65', 'less_than_65_test_X', 'less_than_65_test_y', 'test_ICUType_less_than_65', 'ICUType_1_test_X', 'ICUType_1_test_y', 'test_ICUType_1', 'ICUType_2_test_X', 'ICUType_2_test_y', 'test_ICUType_2', 'ICUType_3_test_X', 'ICUType_3_test_y', 'test_ICUType_3', 'ICUType_4_test_X', 'ICUType_4_test_y', 'test_ICUType_4', 'classificacao_undefined_test_X', 'classificacao_undefined_test_y', 'test_ICUType_classificacao_undefined', 'classificacao_baixo_peso_test_X', 'classificacao_baixo_peso_test_y', 

#### MinMaxScaler

In [2]:
set_random_seed()

# Load the PhysioNet-2012 dataset
physionet2012_dataset = preprocess_physionet2012(subset="all", rate=0.1, normalization=2)

# Take a look at the generated PhysioNet-2012 dataset, you'll find that everything has been prepared for you,
# data splitting, normalization, additional artificially-missing values for evaluation, etc.
print(physionet2012_dataset.keys())

2025-02-10 21:31:04 [INFO]: Have set the random seed as 2022 for numpy and pytorch.
2025-02-10 21:31:04 [INFO]: You're using dataset physionet_2012, please cite it properly in your work. You can find its reference information at the below link: 
https://github.com/WenjieDu/TSDB/tree/main/dataset_profiles/physionet_2012
2025-02-10 21:31:04 [INFO]: Dataset physionet_2012 has already been downloaded. Processing directly...
2025-02-10 21:31:04 [INFO]: Dataset physionet_2012 has already been cached. Loading from cache directly...
2025-02-10 21:31:04 [INFO]: Loaded successfully!
2025-02-10 21:31:19 [WARNING]: Note that physionet_2012 has sparse observations in the time series, hence we don't add additional missing values to the training dataset. 
2025-02-10 21:31:19 [WARNING]: Note that physionet_2012 has sparse observations in the time series, hence we don't add additional missing values to the training dataset. 
2025-02-10 21:31:20 [INFO]: 68807 values masked out in the val set as ground t

dict_keys(['n_classes', 'n_steps', 'n_features', 'scaler', 'train_X', 'train_y', 'train_ICUType', 'val_X', 'val_y', 'val_ICUType', 'test_X', 'test_y', 'test_ICUType', 'female_gender_test_X', 'female_gender_test_y', 'test_ICUType_female_gender', 'male_gender_test_X', 'male_gender_test_y', 'test_ICUType_male_gender', 'undefined_gender_test_X', 'undefined_gender_test_y', 'test_ICUType_undefined_gender', 'more_than_or_equal_to_65_test_X', 'more_than_or_equal_to_65_test_y', 'test_ICUType_more_than_or_equal_to_65', 'less_than_65_test_X', 'less_than_65_test_y', 'test_ICUType_less_than_65', 'ICUType_1_test_X', 'ICUType_1_test_y', 'test_ICUType_1', 'ICUType_2_test_X', 'ICUType_2_test_y', 'test_ICUType_2', 'ICUType_3_test_X', 'ICUType_3_test_y', 'test_ICUType_3', 'ICUType_4_test_X', 'ICUType_4_test_y', 'test_ICUType_4', 'classificacao_undefined_test_X', 'classificacao_undefined_test_y', 'test_ICUType_classificacao_undefined', 'classificacao_baixo_peso_test_X', 'classificacao_baixo_peso_test_y', 

In [3]:
# assemble the datasets for training
dataset_for_training = {
    "X": physionet2012_dataset['train_X'],
}
# assemble the datasets for validation
dataset_for_validating = {
    "X": physionet2012_dataset['val_X'],
    "X_ori": physionet2012_dataset['val_X_ori'],
}

dataset_for_testing_ori = {
    "X_ori": physionet2012_dataset['test_X_ori'],
    "female_gender_test_X_ori": physionet2012_dataset['female_gender_test_X_ori'],
    "male_gender_test_X_ori": physionet2012_dataset['male_gender_test_X_ori'],
    "undefined_gender_test_X_ori": physionet2012_dataset['undefined_gender_test_X_ori'],
    "more_than_or_equal_to_65_test_X_ori":  physionet2012_dataset['more_than_or_equal_to_65_test_X_ori'],
    "less_than_65_test_X_ori": physionet2012_dataset['less_than_65_test_X_ori'],
    "ICUType_1_test_X_ori": physionet2012_dataset['ICUType_1_test_X_ori'],
    "ICUType_2_test_X_ori": physionet2012_dataset['ICUType_2_test_X_ori'],
    "ICUType_3_test_X_ori": physionet2012_dataset['ICUType_3_test_X_ori'],
    "ICUType_4_test_X_ori": physionet2012_dataset['ICUType_4_test_X_ori'],
    "classificacao_undefined_test_X_ori": physionet2012_dataset['classificacao_undefined_test_X_ori'],
    "classificacao_baixo_peso_test_X_ori": physionet2012_dataset['classificacao_baixo_peso_test_X_ori'],
    "classificacao_normal_peso_test_X_ori": physionet2012_dataset['classificacao_normal_peso_test_X_ori'],
    "classificacao_sobrepeso_test_X_ori": physionet2012_dataset['classificacao_sobrepeso_test_X_ori'],
    "classificacao_obesidade_1_test_X_ori": physionet2012_dataset['classificacao_obesidade_1_test_X_ori'],
    "classificacao_obesidade_2_test_X_ori": physionet2012_dataset['classificacao_obesidade_2_test_X_ori'],
    "classificacao_obesidade_3_test_X_ori": physionet2012_dataset['classificacao_obesidade_3_test_X_ori']
}

# assemble the datasets for test
dataset_for_testing = {
    "X": physionet2012_dataset['test_X'],
    "female_gender_test_X": physionet2012_dataset['female_gender_test_X'],
    "male_gender_test_X": physionet2012_dataset['male_gender_test_X'],
    "undefined_gender_test_X": physionet2012_dataset['undefined_gender_test_X'],
    "more_than_or_equal_to_65_test_X":  physionet2012_dataset['more_than_or_equal_to_65_test_X'],
    "less_than_65_test_X": physionet2012_dataset['less_than_65_test_X'],
    "ICUType_1_test_X": physionet2012_dataset['ICUType_1_test_X'],
    "ICUType_2_test_X": physionet2012_dataset['ICUType_2_test_X'],
    "ICUType_3_test_X": physionet2012_dataset['ICUType_3_test_X'],
    "ICUType_4_test_X": physionet2012_dataset['ICUType_4_test_X'],
    "classificacao_undefined_test_X": physionet2012_dataset['classificacao_undefined_test_X'],
    "classificacao_baixo_peso_test_X": physionet2012_dataset['classificacao_baixo_peso_test_X'],
    "classificacao_normal_peso_test_X": physionet2012_dataset['classificacao_normal_peso_test_X'],
    "classificacao_sobrepeso_test_X": physionet2012_dataset['classificacao_sobrepeso_test_X'],
    "classificacao_obesidade_1_test_X": physionet2012_dataset['classificacao_obesidade_1_test_X'],
    "classificacao_obesidade_2_test_X": physionet2012_dataset['classificacao_obesidade_2_test_X'],
    "classificacao_obesidade_3_test_X": physionet2012_dataset['classificacao_obesidade_3_test_X']
}

## calculate the mask to indicate the ground truth positions in test_X_ori, will be used by metric funcs to evaluate models
test_X_indicating_mask = []
test_X_ori = []
for i, j in zip(dataset_for_testing_ori.values(), dataset_for_testing.values()):
    test_X_indicating_mask.append(np.isnan(i) ^ np.isnan(j))
    test_X_ori.append(np.nan_to_num(i))# metric functions do not accpet input with NaNs, hence fill NaNs with 0


In [4]:
test_X_indicating_mask_variable = []
test_X_ori_variable = []
for i in range(len(test_X_indicating_mask)):
    test_X_indicating_mask_variable.append(test_X_indicating_mask[i].reshape(37, len(test_X_indicating_mask[i]) * 48))
    test_X_ori_variable.append(test_X_ori[i].reshape(37, len(test_X_ori[i]) * 48))

## Inicialize the models

#### SAITS

In [5]:
saits = SAITS(
    n_steps=physionet2012_dataset['n_steps'],
    n_features=physionet2012_dataset['n_features'],
    n_layers=1,
    d_model=256,
    d_ffn=128,
    n_heads=4,
    d_k=64,
    d_v=64,
    dropout=0.1,
    ORT_weight=1,  # you can adjust the weight values of arguments ORT_weight
    # and MIT_weight to make the SAITS model focus more on one task. Usually you can just leave them to the default values, i.e. 1.
    MIT_weight=1,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/saits",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-02-10 21:31:44 [INFO]: No given device, using default device: cuda
2025-02-10 21:31:44 [INFO]: Model files will be saved to tutorial_results/imputation/saits/20250210_T213144
2025-02-10 21:31:44 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/saits/20250210_T213144/tensorboard
2025-02-10 21:31:44 [INFO]: SAITS initialized with the given hyperparameters, the number of trainable parameters: 720,182


In [5]:
saits = SAITS(
    n_steps=physionet2012_dataset['n_steps'],
    n_features=physionet2012_dataset['n_features'],
    n_layers=1,
    d_model=256,
    d_ffn=128,
    n_heads=4,
    d_k=64,
    d_v=64,
    dropout=0.1,
    ORT_weight=1,  # you can adjust the weight values of arguments ORT_weight
    # and MIT_weight to make the SAITS model focus more on one task. Usually you can just leave them to the default values, i.e. 1.
    MIT_weight=1,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-02-06 23:24:40 [INFO]: No given device, using default device: cuda
2025-02-06 23:24:40 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.


2025-02-06 23:24:40 [INFO]: SAITS initialized with the given hyperparameters, the number of trainable parameters: 720,182


#### BRITS

In [13]:
brits = BRITS(
    n_steps=physionet2012_dataset['n_steps'],
    n_features=physionet2012_dataset['n_features'],
    rnn_hidden_size=128,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/brits",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-02-10 21:34:59 [INFO]: No given device, using default device: cuda
2025-02-10 21:34:59 [INFO]: Model files will be saved to tutorial_results/imputation/brits/20250210_T213459
2025-02-10 21:34:59 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/brits/20250210_T213459/tensorboard
2025-02-10 21:34:59 [INFO]: BRITS initialized with the given hyperparameters, the number of trainable parameters: 239,344


In [6]:
brits = BRITS(
    n_steps=physionet2012_dataset['n_steps'],
    n_features=physionet2012_dataset['n_features'],
    rnn_hidden_size=128,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-02-06 23:24:47 [INFO]: No given device, using default device: cuda
2025-02-06 23:24:47 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-02-06 23:24:47 [INFO]: BRITS initialized with the given hyperparameters, the number of trainable parameters: 239,344


#### USGAN

In [24]:
us_gan = USGAN(
    n_steps=physionet2012_dataset['n_steps'],
    n_features=physionet2012_dataset['n_features'],
    rnn_hidden_size=256,
    lambda_mse=1,
    dropout=0.1,
    G_steps=1,
    D_steps=1,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    G_optimizer=Adam(lr=1e-3),
    D_optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/us_gan",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-02-05 14:47:07 [INFO]: No given device, using default device: cuda
2025-02-05 14:47:07 [INFO]: Model files will be saved to tutorial_results/imputation/us_gan/20250205_T144707
2025-02-05 14:47:07 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/us_gan/20250205_T144707/tensorboard
2025-02-05 14:47:07 [INFO]: USGAN initialized with the given hyperparameters, the number of trainable parameters: 1,258,517


In [7]:
us_gan = USGAN(
    n_steps=physionet2012_dataset['n_steps'],
    n_features=physionet2012_dataset['n_features'],
    rnn_hidden_size=256,
    lambda_mse=1,
    dropout=0.1,
    G_steps=1,
    D_steps=1,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    G_optimizer=Adam(lr=1e-3),
    D_optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-02-06 23:24:55 [INFO]: No given device, using default device: cuda
2025-02-06 23:24:55 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-02-06 23:24:55 [INFO]: USGAN initialized with the given hyperparameters, the number of trainable parameters: 1,258,517


#### GPVAE

In [37]:
gp_vae = GPVAE(
    n_steps=physionet2012_dataset['n_steps'],
    n_features=physionet2012_dataset['n_features'],
    latent_size=37,
    encoder_sizes=(128,128),
    decoder_sizes=(256,256),
    kernel="cauchy",
    beta=0.2,
    M=1,
    K=1,
    sigma=1.005,
    length_scale=7.0,
    kernel_scales=1,
    window_size=24,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/gp_vae",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-02-05 16:04:27 [INFO]: No given device, using default device: cuda
2025-02-05 16:04:27 [INFO]: Model files will be saved to tutorial_results/imputation/gp_vae/20250205_T160427
2025-02-05 16:04:27 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/gp_vae/20250205_T160427/tensorboard
2025-02-05 16:04:27 [INFO]: GPVAE initialized with the given hyperparameters, the number of trainable parameters: 229,652


In [5]:
gp_vae = GPVAE(
    n_steps=physionet2012_dataset['n_steps'],
    n_features=physionet2012_dataset['n_features'],
    latent_size=37,
    encoder_sizes=(128,128),
    decoder_sizes=(256,256),
    kernel="cauchy",
    beta=0.2,
    M=1,
    K=1,
    sigma=1.005,
    length_scale=7.0,
    kernel_scales=1,
    window_size=24,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-02-10 22:29:48 [INFO]: No given device, using default device: cuda
2025-02-10 22:29:48 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.


2025-02-10 22:29:48 [INFO]: GPVAE initialized with the given hyperparameters, the number of trainable parameters: 229,652


#### MRNN

In [5]:
mrnn = MRNN(
    n_steps=physionet2012_dataset['n_steps'],
    n_features=physionet2012_dataset['n_features'],
    rnn_hidden_size=128,

    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/mrnn",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-02-05 16:50:40 [INFO]: No given device, using default device: cuda
2025-02-05 16:50:40 [INFO]: Model files will be saved to tutorial_results/imputation/mrnn/20250205_T165040
2025-02-05 16:50:40 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/mrnn/20250205_T165040/tensorboard


2025-02-05 16:50:41 [INFO]: MRNN initialized with the given hyperparameters, the number of trainable parameters: 107,951


In [9]:
mrnn = MRNN(
    n_steps=physionet2012_dataset['n_steps'],
    n_features=physionet2012_dataset['n_features'],
    rnn_hidden_size=128,

    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-02-06 23:25:11 [INFO]: No given device, using default device: cuda
2025-02-06 23:25:11 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-02-06 23:25:11 [INFO]: MRNN initialized with the given hyperparameters, the number of trainable parameters: 107,951


## Train/Load the models

#### SAITS

In [6]:
# train the model on the training set, and validate it on the validating set to select the best model for testing in the next step
saits.fit(train_set=dataset_for_training, val_set=dataset_for_validating)

2025-02-10 21:32:13 [INFO]: Epoch 001 - training loss: 0.1661, validation loss: 0.0086
2025-02-10 21:32:13 [INFO]: Saved the model to tutorial_results/imputation/saits/20250210_T213144/SAITS_epoch1_loss0.00861132900075366.pypots
2025-02-10 21:32:17 [INFO]: Epoch 002 - training loss: 0.1120, validation loss: 0.0072
2025-02-10 21:32:17 [INFO]: Saved the model to tutorial_results/imputation/saits/20250210_T213144/SAITS_epoch2_loss0.007171140192076564.pypots
2025-02-10 21:32:22 [INFO]: Epoch 003 - training loss: 0.0949, validation loss: 0.0054
2025-02-10 21:32:22 [INFO]: Saved the model to tutorial_results/imputation/saits/20250210_T213144/SAITS_epoch3_loss0.0054213163792155685.pypots
2025-02-10 21:32:27 [INFO]: Epoch 004 - training loss: 0.0820, validation loss: 0.0050
2025-02-10 21:32:27 [INFO]: Saved the model to tutorial_results/imputation/saits/20250210_T213144/SAITS_epoch4_loss0.004954955098219216.pypots
2025-02-10 21:32:32 [INFO]: Epoch 005 - training loss: 0.0727, validation loss: 

#### Load - Standard Scaler

In [10]:
saits.load("tutorial_results/imputation/saits/20250204_T160556/SAITS.pypots")

/usr/local/lib/python3.11/dist-packages/pypots/base.py:325: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_model = torch.load(path, map_location=self.device)
2025-02-0

#### Load - Min/Max Scaler

In [ ]:
saits.load("tutorial_results/imputation/saits/20250210_T213144/SAITS.pypots")

#### BRITS

In [14]:
# train the model on the training set, and validate it on the validating set to select the best model for testing in the next step
brits.fit(train_set=dataset_for_training, val_set=dataset_for_validating)

2025-02-10 21:37:11 [INFO]: Epoch 001 - training loss: 0.1748, validation loss: 0.0083
2025-02-10 21:37:11 [INFO]: Saved the model to tutorial_results/imputation/brits/20250210_T213459/BRITS_epoch1_loss0.00834462307393551.pypots
2025-02-10 21:38:52 [INFO]: Epoch 002 - training loss: 0.0984, validation loss: 0.0060
2025-02-10 21:38:52 [INFO]: Saved the model to tutorial_results/imputation/brits/20250210_T213459/BRITS_epoch2_loss0.005978443135973066.pypots
2025-02-10 21:40:33 [INFO]: Epoch 003 - training loss: 0.0821, validation loss: 0.0052
2025-02-10 21:40:33 [INFO]: Saved the model to tutorial_results/imputation/brits/20250210_T213459/BRITS_epoch3_loss0.005158932743749271.pypots
2025-02-10 21:42:12 [INFO]: Epoch 004 - training loss: 0.0753, validation loss: 0.0046
2025-02-10 21:42:12 [INFO]: Saved the model to tutorial_results/imputation/brits/20250210_T213459/BRITS_epoch4_loss0.004613198402027289.pypots
2025-02-10 21:43:54 [INFO]: Epoch 005 - training loss: 0.0707, validation loss: 0

#### Load - Standard Scaler

In [11]:
brits.load("tutorial_results/imputation/brits/20250204_T162108/BRITS.pypots")

2025-02-06 23:25:24 [INFO]: Model loaded successfully from tutorial_results/imputation/brits/20250204_T162108/BRITS.pypots


#### Load - Min/Max Scaler

In [10]:
brits.load("tutorial_results/imputation/brits/20250210_T213459/BRITS.pypots")

NameError: name 'brits' is not defined

#### USGAN

In [25]:
us_gan.fit(train_set=dataset_for_training, val_set=dataset_for_validating)

2025-02-05 14:52:52 [INFO]: Epoch 001 - generator training loss: 0.4299, discriminator training loss: 0.1869, validation loss: 6.7646
2025-02-05 14:52:52 [INFO]: Saved the model to tutorial_results/imputation/us_gan/20250205_T144707/USGAN_epoch1_loss6.764622490356365.pypots
2025-02-05 14:56:00 [INFO]: Epoch 002 - generator training loss: 0.3612, discriminator training loss: 0.0552, validation loss: 6.7262
2025-02-05 14:56:00 [INFO]: Saved the model to tutorial_results/imputation/us_gan/20250205_T144707/USGAN_epoch2_loss6.7262225342293585.pypots
2025-02-05 14:59:29 [INFO]: Epoch 003 - generator training loss: 0.3365, discriminator training loss: 0.0370, validation loss: 6.6997
2025-02-05 14:59:29 [INFO]: Saved the model to tutorial_results/imputation/us_gan/20250205_T144707/USGAN_epoch3_loss6.699678348998229.pypots
2025-02-05 15:02:31 [INFO]: Epoch 004 - generator training loss: 0.3192, discriminator training loss: 0.0312, validation loss: 6.6862
2025-02-05 15:02:31 [INFO]: Saved the mo

In [12]:
us_gan.load("tutorial_results/imputation/us_gan/20250205_T144707/USGAN.pypots")

2025-02-06 23:25:28 [INFO]: Model loaded successfully from tutorial_results/imputation/us_gan/20250205_T144707/USGAN.pypots


#### GPVAE

In [38]:
gp_vae.fit(train_set=dataset_for_training, val_set=dataset_for_validating)

2025-02-05 16:06:08 [INFO]: Epoch 001 - training loss: 26076.3818, validation loss: 7.0346
2025-02-05 16:06:08 [INFO]: Saved the model to tutorial_results/imputation/gp_vae/20250205_T160427/GPVAE_epoch1_loss7.034577655792236.pypots
2025-02-05 16:06:14 [INFO]: Epoch 002 - training loss: 22871.6474, validation loss: 6.9894
2025-02-05 16:06:14 [INFO]: Saved the model to tutorial_results/imputation/gp_vae/20250205_T160427/GPVAE_epoch2_loss6.989437110722065.pypots
2025-02-05 16:06:21 [INFO]: Epoch 003 - training loss: 22836.6079, validation loss: 6.9642
2025-02-05 16:06:21 [INFO]: Saved the model to tutorial_results/imputation/gp_vae/20250205_T160427/GPVAE_epoch3_loss6.964227601885796.pypots
2025-02-05 16:06:27 [INFO]: Epoch 004 - training loss: 22828.9871, validation loss: 6.9665
2025-02-05 16:06:33 [INFO]: Epoch 005 - training loss: 22820.1492, validation loss: 6.9349
2025-02-05 16:06:33 [INFO]: Saved the model to tutorial_results/imputation/gp_vae/20250205_T160427/GPVAE_epoch5_loss6.9349

In [6]:
gp_vae.load("tutorial_results/imputation/gp_vae/20250205_T160427/GPVAE.pypots")

/usr/local/lib/python3.11/dist-packages/pypots/base.py:324: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_model = torch.load(path, map_location=self.device)
2025-02-1

#### MRNN

In [6]:
mrnn.fit(train_set=dataset_for_training, val_set=dataset_for_validating)

2025-02-05 16:51:24 [INFO]: Epoch 001 - training loss: 0.7654, validation loss: 7.3141
2025-02-05 16:51:24 [INFO]: Saved the model to tutorial_results/imputation/mrnn/20250205_T165040/MRNN_epoch1_loss7.314062108596166.pypots
2025-02-05 16:51:40 [INFO]: Epoch 002 - training loss: 0.5383, validation loss: 7.2726
2025-02-05 16:51:40 [INFO]: Saved the model to tutorial_results/imputation/mrnn/20250205_T165040/MRNN_epoch2_loss7.272605112195015.pypots
2025-02-05 16:51:57 [INFO]: Epoch 003 - training loss: 0.4978, validation loss: 7.2576
2025-02-05 16:51:57 [INFO]: Saved the model to tutorial_results/imputation/mrnn/20250205_T165040/MRNN_epoch3_loss7.257625941435496.pypots
2025-02-05 16:52:14 [INFO]: Epoch 004 - training loss: 0.4734, validation loss: 7.2509
2025-02-05 16:52:14 [INFO]: Saved the model to tutorial_results/imputation/mrnn/20250205_T165040/MRNN_epoch4_loss7.250920605659485.pypots
2025-02-05 16:52:31 [INFO]: Epoch 005 - training loss: 0.4672, validation loss: 7.2455
2025-02-05 16

In [14]:
mrnn.load("tutorial_results/imputation/mrnn/20250205_T165040/MRNN.pypots")

2025-02-06 23:25:34 [INFO]: Model loaded successfully from tutorial_results/imputation/mrnn/20250205_T165040/MRNN.pypots


## Testing stage

#### SAITS

In [7]:
saits_imputation = []
for value in  dataset_for_testing.values():
   _dict = {'X':value}
   saits_results = saits.predict(_dict)
   saits_imputation.append(saits_results["imputation"])


In [8]:
saits_imputation_variable = []
for i in range(len(saits_imputation)):
    saits_imputation_variable.append(saits_imputation[i].reshape(37, len(saits_imputation[i]) * 48))


#### BRITS

In [15]:
brits_imputation = []
for value in dataset_for_testing.values():
    _dict = {'X':value}
    brits_results = brits.predict(_dict)
    brits_imputation.append(brits_results["imputation"])

In [16]:
brits_imputation_variable = []
for i in range(len(brits_imputation)):
    brits_imputation_variable.append(brits_imputation[i].reshape(37, len(brits_imputation[i]) * 48))

#### USGAN

In [19]:
usgan_imputation = []
for value in  dataset_for_testing.values():
   _dict = {'X':value}
   usgan_results = us_gan.predict(_dict)
   usgan_imputation.append(usgan_results["imputation"])

In [20]:
usgan_imputation_variable = []
for i in range(len(usgan_imputation)):
    usgan_imputation_variable.append(usgan_imputation[i].reshape(37, len(usgan_imputation[i]) * 48))

#### GPVAE

In [7]:
gpvae_imputation = []
for value in  dataset_for_testing.values():
   _dict = {'X':value}
   gpvae_results = gp_vae.predict(_dict)
   gpvae_imputation.append(gpvae_results["imputation"])

In [9]:
gpvae_imputation_variable = []
for i in range(len(gpvae_imputation)):
    gpvae_imputation_variable.append(gpvae_imputation[i].reshape(37, len(gpvae_imputation[i]) * 48))

#### MRNN

In [23]:
mrnn_imputation = []
for value in  dataset_for_testing.values():
   _dict = {'X':value}
   mrnn_results = mrnn.predict(_dict)
   mrnn_imputation.append(mrnn_results["imputation"])

In [24]:
mrnn_imputation_variable = []
for i in range(len(mrnn_imputation)):
    mrnn_imputation_variable.append(mrnn_imputation[i].reshape(37, len(mrnn_imputation[i]) * 48))

## Calculate mean absolute error

#### SAITS

In [9]:
testing_mae_saits_append_subgroups = []
testing_mae_saits_append_variables = []
for i in range(len(saits_imputation_variable)):
    for j in range(len(saits_imputation_variable[i])):
        testing_mae_saits_append_variables.append(calc_mae(saits_imputation_variable[i][j], test_X_ori_variable[i][j], test_X_indicating_mask_variable[i][j]))
    testing_mae_saits_append_subgroups.append(testing_mae_saits_append_variables)
    testing_mae_saits_append_variables = [] 

#### BRITS

In [17]:
testing_mae_brits_append_subgroups = []
testing_mae_brits_append_variables = []
for i in range(len(brits_imputation_variable)):
    for j in range(len(brits_imputation_variable[i])):
      testing_mae_brits_append_variables.append(calc_mae(brits_imputation_variable[i][j], test_X_ori_variable[i][j], test_X_indicating_mask_variable[i][j]))
    testing_mae_brits_append_subgroups.append(testing_mae_brits_append_variables)
    testing_mae_brits_append_variables = []

#### USGAN

In [27]:
testing_mae_usgan_append_subgroups = []
testing_mae_usgan_append_variables = []
for i in range(len(usgan_imputation_variable)):
    for j in range(len(usgan_imputation_variable[i])):
        testing_mae_usgan_append_variables.append(calc_mae(usgan_imputation_variable[i][j], test_X_ori_variable[i][j], test_X_indicating_mask_variable[i][j]))
    testing_mae_usgan_append_subgroups.append(testing_mae_usgan_append_variables)
    testing_mae_usgan_append_variables = []

#### GPVAE

In [28]:
testing_mae_gpvae_append_subgroups = []
testing_mae_gpvae_append_variables = []
for i in range(len(gpvae_imputation_variable)):
    for j in range(len(gpvae_imputation_variable[i])):
        testing_mae_gpvae_append_variables.append(calc_mae(gpvae_imputation_variable[i][j], test_X_ori_variable[i][j], test_X_indicating_mask_variable[i][j]))
    testing_mae_gpvae_append_subgroups.append(testing_mae_gpvae_append_variables)
    testing_mae_gpvae_append_variables = []

#### MRNN

In [29]:
testing_mae_mrnn_append_subgroups = []
testing_mae_mrnn_append_variables = []
for i in range(len(mrnn_imputation_variable)):
    for j in range(len(mrnn_imputation_variable[i])):
        testing_mae_mrnn_append_variables.append(calc_mae(mrnn_imputation_variable[i][j], test_X_ori_variable[i][j], test_X_indicating_mask_variable[i][j]))
    testing_mae_mrnn_append_subgroups.append(testing_mae_mrnn_append_variables)
    testing_mae_mrnn_append_variables = []

## Results mean absolute error

In [10]:
subgroups = ["General", "Female", "Male", "Undefined Gender", "+65", "-65", "ICUType 1", "ICUType 2", "ICUType 3", "ICUType 4", "Undefined classification", "Low Weight", "Normal Weight", "Overweight", "Obesity 1", "Obesity 2", "Obesity 3"]

In [11]:
variables = ["ALP", "ALT", "AST", "Albumin", "BUN", "Bilirubin", "Cholesterol", "Creatinine", "DiasABP", "FiO2", "GCS", "Glucose", "HCO3", "HCT", "HR", "K", "Lactate", "MAP", "MechVent", "Mg", "NIDiasABP", "NIMAP", "NISysABP", "Na", "PaCO2", "PaO2", "Platelets", "RespRate", "SaO2", "SysABP", "Temp", "TroponinI", "TroponinT", "Urine", "WBC", "Weight", "Ph"]

#### SAITS

In [12]:
print("SAITS - MAE")
print("************")
for i in range(len(subgroups)): 
    print(subgroups[i]) 
    print("-------------")
    for j in range(len(variables)):
        print(variables[j], ":" ,testing_mae_saits_append_subgroups[i][j])

SAITS - MAE
************
General
-------------
ALP : 0.03104436406688503
ALT : 0.028170038474726548
AST : 0.03196371920284396
Albumin : 0.03214313369916851
BUN : 0.029658829345883125
Bilirubin : 0.029296447671938196
Cholesterol : 0.03053975247482222
Creatinine : 0.031680863085205166
DiasABP : 0.027608199296117835
FiO2 : 0.029297089283156764
GCS : 0.03296321559771377
Glucose : 0.031347762229944114
HCO3 : 0.031868699024913724
HCT : 0.029033698823755963
HR : 0.028910497775058167
K : 0.028932415249945338
Lactate : 0.029528962856012284
MAP : 0.030834777847436306
MechVent : 0.02951229421593849
Mg : 0.033016768728603076
NIDiasABP : 0.02907365682914185
NIMAP : 0.02980853449343487
NISysABP : 0.030117783059762263
Na : 0.030350306928081234
PaCO2 : 0.028205603472151276
PaO2 : 0.03066153177029523
Platelets : 0.030168870919250548
RespRate : 0.030673072030405077
SaO2 : 0.03010541785488216
SysABP : 0.02965602938524825
Temp : 0.028850114467301468
TroponinI : 0.02876803508410699
TroponinT : 0.0277444072

In [33]:
df_saits_mae = pd.DataFrame(variables)

for i in range(len(subgroups)):
    df_saits_mae[subgroups[i]] = testing_mae_saits_append_subgroups[i]

In [34]:
df_saits_mae

,0,General,Female,Male,Undefined Gender,+65,-65,ICUType 1,ICUType 2,ICUType 3,ICUType 4,Undefined classification,Low Weight,Normal Weight,Overweight,Obesity 1,Obesity 2,Obesity 3
0,ALP,0.246029,0.220452,0.281318,0.024085,0.254108,0.286183,0.291996,0.220202,0.261112,0.264622,0.292983,0.277895,0.247686,0.237087,0.291813,0.180335,0.239991
1,ALT,0.220431,0.231447,0.222804,0.000000,0.254721,0.251318,0.278319,0.228516,0.228215,0.239234,0.248507,0.326892,0.210455,0.191214,0.277488,0.329962,0.228294
2,AST,0.262308,0.263395,0.237008,0.295356,0.248577,0.261737,0.274582,0.216824,0.246338,0.239527,0.266911,0.324861,0.244048,0.266117,0.213830,0.227629,0.178151
3,Albumin,0.302933,0.245631,0.308385,0.256960,0.225847,0.242713,0.218852,0.215799,0.271407,0.238350,0.241231,0.368676,0.253718,0.222560,0.287282,0.245201,0.181224
4,BUN,0.236598,0.224053,0.245069,0.038733,0.277838,0.235693,0.308771,0.204856,0.252615,0.247639,0.271124,0.281561,0.237951,0.256407,0.246900,0.205267,0.306612
5,Bilirubin,0.231291,0.250564,0.265393,0.098811,0.267808,0.258753,0.256349,0.205388,0.303023,0.255608,0.265551,0.239788,0.244810,0.222906,0.190586,0.242239,0.210118
6,Cholesterol,0.251680,0.231705,0.233628,0.169514,0.234425,0.274246,0.292118,0.201527,0.292166,0.258185,0.268551,0.178686,0.185244,0.216891,0.247719,0.277958,0.250856
7,Creatinine,0.255735,0.263072,0.283228,0.242628,0.258071,0.278139,0.214152,0.231790,0.269252,0.251534,0.281256,0.188572,0.222748,0.263322,0.231723,0.248706,0.189410
8,DiasABP,0.236598,0.265557,0.260386,0.129530,0.238133,0.369640,0.231949,0.207031,0.286422,0.259389,0.293405,0.250004,0.206029,0.202719,0.227379,0.176758,0.207900
9,FiO2,0.243917,0.263631,0.248451,0.118587,0.236096,0.279574,0.257063,0.218261,0.281995,0.237350,0.246835,0.178762,0.210799,0.248204,0.257538,0.231925,0.243773


Minimum MAE value in each subgroup

In [35]:
for i in range(len(subgroups)):
    value = df_saits_mae[subgroups[i]].min()
    print(subgroups[i])
    for j in range(len(variables)):
        if(df_saits_mae[subgroups[i]][j] == value):
            var = variables[j]
    print(var)
    print(value)        
    print("--------------------")
        

General
ALT
0.22043134772111558
--------------------
Female
NIDiasABP
0.21625072772560813
--------------------
Male
PaCO2
0.2108230852051344
--------------------
Undefined Gender
Ph
0.0
--------------------
+65
PaO2
0.2052877660438494
--------------------
-65
RespRate
0.2195340976855963
--------------------
ICUType 1
Creatinine
0.21415215930457915
--------------------
ICUType 2
Ph
0.19352872186986245
--------------------
ICUType 3
PaCO2
0.2245388907288932
--------------------
ICUType 4
HCT
0.22102052107602238
--------------------
Undefined classification
TroponinT
0.22312300711184532
--------------------
Low Weight
K
0.1351692709542507
--------------------
Normal Weight
Cholesterol
0.1852442869645915
--------------------
Overweight
MAP
0.18507979199527627
--------------------
Obesity 1
Platelets
0.16087222354644645
--------------------
Obesity 2
WBC
0.1394584956765725
--------------------
Obesity 3
AST
0.17815062101412324
--------------------


Maximum MAE value in each subgroup

In [37]:
for i in range(len(subgroups)):
    value = df_saits_mae[subgroups[i]].max()
    print(subgroups[i])
    for j in range(len(variables)):
        if(df_saits_mae[subgroups[i]][j] == value):
            var = variables[j]
    print(var)
    print(value)        
    print("--------------------")    

General
Albumin
0.30293279432474296
--------------------
Female
Urine
0.30370237839850694
--------------------
Male
Albumin
0.308384918688299
--------------------
Undefined Gender
HCT
0.5138421868725681
--------------------
+65
BUN
0.2778384531342736
--------------------
-65
DiasABP
0.3696399065245959
--------------------
ICUType 1
PaO2
0.3504441683775851
--------------------
ICUType 2
NIMAP
0.2556318391589281
--------------------
ICUType 3
PaO2
0.30687231283103145
--------------------
ICUType 4
MAP
0.3212451658407674
--------------------
Undefined classification
HCO3
0.29353148208906726
--------------------
Low Weight
NIMAP
0.6064582243175376
--------------------
Normal Weight
Glucose
0.2738133891557834
--------------------
Overweight
WBC
0.27807657394047125
--------------------
Obesity 1
HCO3
0.8932678493057511
--------------------
Obesity 2
SaO2
0.4265312900933624
--------------------
Obesity 3
NIMAP
0.4806281971702897
--------------------


#### BRITS

In [18]:
print("BRITS - MAE")
print("************")
for i in range(len(subgroups)):
    print(subgroups[i]) 
    print("-------------")
    for j in range(len(variables)):
        print(variables[j], ":" ,testing_mae_brits_append_subgroups[i][j])

BRITS - MAE
************
General
-------------
ALP : 0.02714361165444072
ALT : 0.023480500236732458
AST : 0.02872805607407688
Albumin : 0.02660760645174079
BUN : 0.025555022701653395
Bilirubin : 0.025085490378176457
Cholesterol : 0.026067523792760033
Creatinine : 0.026274485669448455
DiasABP : 0.023210803510269006
FiO2 : 0.024787476533681622
GCS : 0.02815564995991108
Glucose : 0.02558339101854918
HCO3 : 0.025903053240614073
HCT : 0.025094551013020577
HR : 0.02439231563245925
K : 0.02443751426771262
Lactate : 0.02509530837204354
MAP : 0.0253123337344026
MechVent : 0.025884273229850978
Mg : 0.027852336603015535
NIDiasABP : 0.02431355283328725
NIMAP : 0.025348164389083157
NISysABP : 0.026094541170267913
Na : 0.024826197105273985
PaCO2 : 0.02344621102572964
PaO2 : 0.02580412300815957
Platelets : 0.025184702786608178
RespRate : 0.025502672447018395
SaO2 : 0.0266654004802727
SysABP : 0.025151581834592308
Temp : 0.025333080413934686
TroponinI : 0.02481646924598073
TroponinT : 0.02332166459826

In [39]:
df_brits_mae = pd.DataFrame(variables)

for i in range(len(subgroups)):
    df_brits_mae[subgroups[i]] = testing_mae_brits_append_subgroups[i]

In [40]:
df_brits_mae 

,0,General,Female,Male,Undefined Gender,+65,-65,ICUType 1,ICUType 2,ICUType 3,ICUType 4,Undefined classification,Low Weight,Normal Weight,Overweight,Obesity 1,Obesity 2,Obesity 3
0,ALP,0.270737,0.256535,0.293297,0.329968,0.264696,0.315839,0.306754,0.226954,0.273958,0.271468,0.302521,0.256009,0.249952,0.258540,0.295754,0.216197,0.235523
1,ALT,0.238966,0.233629,0.244945,0.000000,0.276510,0.256375,0.292688,0.235882,0.250922,0.252148,0.281570,0.354939,0.216862,0.219696,0.282570,0.585253,0.224331
2,AST,0.288784,0.309354,0.251649,0.438452,0.269273,0.279676,0.316330,0.218515,0.275573,0.262117,0.281901,0.468715,0.247855,0.287826,0.233795,0.257504,0.206570
3,Albumin,0.321582,0.268955,0.321363,0.418167,0.259164,0.262846,0.263338,0.213691,0.289130,0.238066,0.247420,0.396643,0.267411,0.235706,0.330971,0.242369,0.202276
4,BUN,0.243729,0.233320,0.255147,0.086077,0.298458,0.258437,0.335850,0.245378,0.272088,0.261600,0.301877,0.375226,0.244099,0.266491,0.258967,0.220044,0.267429
5,Bilirubin,0.251635,0.272584,0.292882,0.139311,0.280527,0.276679,0.253002,0.211493,0.318044,0.274579,0.278961,0.242204,0.261701,0.247860,0.209038,0.249435,0.206693
6,Cholesterol,0.253555,0.240798,0.241246,0.173375,0.248621,0.283141,0.288232,0.211427,0.309410,0.267875,0.288611,0.168884,0.199758,0.235638,0.255247,0.314013,0.240939
7,Creatinine,0.272366,0.287540,0.288413,0.238730,0.266385,0.307470,0.229776,0.227727,0.292145,0.252393,0.275201,0.208665,0.256300,0.263794,0.254608,0.249333,0.203272
8,DiasABP,0.250763,0.271252,0.280030,0.090122,0.250686,0.368415,0.258635,0.211360,0.299583,0.270138,0.301323,0.297203,0.223460,0.210306,0.241157,0.223408,0.171138
9,FiO2,0.251453,0.262677,0.255937,0.073339,0.257094,0.287834,0.283073,0.226797,0.298260,0.250877,0.254612,0.181415,0.219981,0.232228,0.271426,0.270714,0.235543


Minimum MAE value in each subgroup

In [41]:
for i in range(len(subgroups)):
    value = df_brits_mae[subgroups[i]].min()
    print(subgroups[i])
    for j in range(len(variables)):
        if(df_brits_mae[subgroups[i]][j] == value):
            var = variables[j]
    print(var)
    print(value)        
    print("--------------------")
        

General
NIDiasABP
0.23742945499040968
--------------------
Female
NIDiasABP
0.2238358455137205
--------------------
Male
PaCO2
0.207682844444775
--------------------
Undefined Gender
Ph
0.0
--------------------
+65
NIMAP
0.2160139074300404
--------------------
-65
RespRate
0.23431749476562846
--------------------
ICUType 1
Platelets
0.22327792083558695
--------------------
ICUType 2
Temp
0.19477174647491813
--------------------
ICUType 3
HCT
0.23571246018322012
--------------------
ICUType 4
HCT
0.22995585027402263
--------------------
Undefined classification
HCT
0.23881988273086038
--------------------
Low Weight
Urine
0.14774463446351938
--------------------
Normal Weight
Cholesterol
0.19975836468216204
--------------------
Overweight
MAP
0.18576449265978498
--------------------
Obesity 1
Mg
0.15740798042800078
--------------------
Obesity 2
WBC
0.15522959001932013
--------------------
Obesity 3
Lactate
0.17065355340995153
--------------------


Maximum MAE value in each subgroup

In [42]:
for i in range(len(subgroups)):
    value = df_brits_mae[subgroups[i]].max()
    print(subgroups[i])
    for j in range(len(variables)):
        if(df_brits_mae[subgroups[i]][j] == value):
            var = variables[j]
    print(var)
    print(value)        
    print("--------------------")
        

General
GCS
0.5404476843954903
--------------------
Female
Urine
0.3300605136654839
--------------------
Male
GCS
0.36500282476200896
--------------------
Undefined Gender
AST
0.43845240134512803
--------------------
+65
GCS
0.31723068628216455
--------------------
-65
DiasABP
0.36841543417646266
--------------------
ICUType 1
PaO2
0.3700832953509951
--------------------
ICUType 2
GCS
0.42147041589578793
--------------------
ICUType 3
Glucose
0.33969703536165907
--------------------
ICUType 4
Urine
0.3230442247268251
--------------------
Undefined classification
WBC
0.30855608846898475
--------------------
Low Weight
NIMAP
0.7751840826066394
--------------------
Normal Weight
GCS
0.4505264138330109
--------------------
Overweight
GCS
0.3038573484678385
--------------------
Obesity 1
HCO3
0.9005014291539718
--------------------
Obesity 2
ALT
0.5852528140030043
--------------------
Obesity 3
NIMAP
0.43202845565556747
--------------------


#### USGAN 

In [43]:
print("USGAN - MAE")
print("************")
for i in range(len(subgroups)):
    print(subgroups[i]) 
    print("-------------")
    for j in range(len(variables)):
        print(variables[j], ":" ,testing_mae_usgan_append_subgroups[i][j])

USGAN - MAE
************
General
-------------
ALP : 0.28327919700514537
ALT : 0.25698387822352164
AST : 0.29381425780834014
Albumin : 0.3316851965822937
BUN : 0.2558014721066809
Bilirubin : 0.26569659912151444
Cholesterol : 0.2717877965417484
Creatinine : 0.2807011303352554
DiasABP : 0.25648366397544786
FiO2 : 0.26940385736786227
GCS : 0.30802102369019674
Glucose : 0.27290957048290815
HCO3 : 0.2796737059846553
HCT : 0.2611459327041813
HR : 0.25871885034518405
K : 0.2614191212323757
Lactate : 0.2785402883573093
MAP : 0.28843045647664245
MechVent : 0.28180003190404546
Mg : 0.2834200527338646
NIDiasABP : 0.2516281229311397
NIMAP : 0.26122135945979064
NISysABP : 0.27605309628950486
Na : 0.27928460154237206
PaCO2 : 0.25462611017019066
PaO2 : 0.27273692419304074
Platelets : 0.2615611081737651
RespRate : 0.26835562858972456
SaO2 : 0.2824451172805901
SysABP : 0.2731640090539184
Temp : 0.27050709716394056
TroponinI : 0.2706214236991907
TroponinT : 0.259756080398352
Urine : 0.28173089277097757


In [44]:
df_usgan_mae = pd.DataFrame(variables)

for i in range(len(subgroups)):
    df_usgan_mae[subgroups[i]] = testing_mae_usgan_append_subgroups[i]

In [45]:
df_usgan_mae

,0,General,Female,Male,Undefined Gender,+65,-65,ICUType 1,ICUType 2,ICUType 3,ICUType 4,Undefined classification,Low Weight,Normal Weight,Overweight,Obesity 1,Obesity 2,Obesity 3
0,ALP,0.283279,0.260204,0.292055,0.639126,0.278950,0.311200,0.324575,0.232860,0.284586,0.274382,0.308400,0.256726,0.278650,0.256221,0.302301,0.219735,0.282214
1,ALT,0.256984,0.266539,0.251540,0.000000,0.297289,0.277807,0.285709,0.251689,0.266937,0.261526,0.291585,0.301074,0.233951,0.228284,0.302541,0.448541,0.250768
2,AST,0.293814,0.303223,0.269565,0.384212,0.279343,0.280914,0.302549,0.236463,0.292150,0.263452,0.298456,0.373478,0.268835,0.295488,0.255971,0.239183,0.225336
3,Albumin,0.331685,0.281944,0.324433,0.283477,0.256423,0.277574,0.263155,0.215530,0.296938,0.245375,0.260768,0.352473,0.268574,0.237813,0.322414,0.232580,0.216323
4,BUN,0.255801,0.252679,0.267172,0.152919,0.299424,0.271188,0.310056,0.249563,0.286176,0.262668,0.302922,0.350310,0.263515,0.274829,0.271573,0.211423,0.304615
5,Bilirubin,0.265697,0.279324,0.293216,0.230283,0.285931,0.266058,0.283463,0.226065,0.332316,0.280361,0.293361,0.288386,0.269280,0.271721,0.254418,0.240187,0.236993
6,Cholesterol,0.271788,0.254231,0.258275,0.284804,0.255848,0.291156,0.310502,0.219850,0.307705,0.280798,0.290638,0.186191,0.214310,0.250294,0.257985,0.307338,0.283547
7,Creatinine,0.280701,0.296907,0.301210,0.221904,0.285033,0.296130,0.244120,0.244509,0.312245,0.275603,0.292766,0.186234,0.280157,0.285964,0.269462,0.279966,0.219875
8,DiasABP,0.256484,0.288802,0.280281,0.180854,0.264324,0.368516,0.293311,0.208421,0.310414,0.278863,0.313247,0.289131,0.223244,0.222269,0.259460,0.240750,0.203677
9,FiO2,0.269404,0.282501,0.275456,0.070404,0.265951,0.308677,0.299912,0.230760,0.305293,0.266495,0.265278,0.196520,0.244525,0.245101,0.278537,0.242282,0.225017


Minimum MAE value in each subgroup

In [46]:
for i in range(len(subgroups)):
    value = df_usgan_mae[subgroups[i]].min()
    print(subgroups[i])
    for j in range(len(variables)):
        if(df_usgan_mae[subgroups[i]][j] == value):
            var = variables[j]
    print(var)
    print(value)        
    print("--------------------")
        

General
NIDiasABP
0.2516281229311397
--------------------
Female
NIDiasABP
0.24216731176568626
--------------------
Male
PaCO2
0.23351034224264478
--------------------
Undefined Gender
Ph
0.0
--------------------
+65
Weight
0.22837968883470952
--------------------
-65
RespRate
0.2510679315358316
--------------------
ICUType 1
Platelets
0.24184056448141877
--------------------
ICUType 2
DiasABP
0.2084214124015174
--------------------
ICUType 3
HCT
0.2527596471830482
--------------------
ICUType 4
Albumin
0.2453749441982267
--------------------
Undefined classification
HCT
0.24869123029565587
--------------------
Low Weight
Urine
0.1768420653889066
--------------------
Normal Weight
Cholesterol
0.21431008457946749
--------------------
Overweight
MAP
0.2195381558157749
--------------------
Obesity 1
Temp
0.187016339458737
--------------------
Obesity 2
WBC
0.16541041512975618
--------------------
Obesity 3
DiasABP
0.20367692515252409
--------------------


Maximum MAE value in each subgroup

In [47]:
for i in range(len(subgroups)):
    value = df_usgan_mae[subgroups[i]].max()
    print(subgroups[i])
    for j in range(len(variables)):
        if(df_usgan_mae[subgroups[i]][j] == value):
            var = variables[j]
    print(var)
    print(value)        
    print("--------------------")
        

General
Albumin
0.3316851965822937
--------------------
Female
Urine
0.3381902288800075
--------------------
Male
GCS
0.3315198371771994
--------------------
Undefined Gender
ALP
0.6391255460494273
--------------------
+65
BUN
0.29942431322994245
--------------------
-65
DiasABP
0.36851604731574583
--------------------
ICUType 1
Na
0.37373211738434214
--------------------
ICUType 2
GCS
0.3574478481949517
--------------------
ICUType 3
Urine
0.34607378180620857
--------------------
ICUType 4
Glucose
0.347358340544533
--------------------
Undefined classification
DiasABP
0.31324712623403267
--------------------
Low Weight
NIMAP
0.6466760653643877
--------------------
Normal Weight
GCS
0.3558426906976595
--------------------
Overweight
GCS
0.3081582051195111
--------------------
Obesity 1
HCO3
0.923044947600093
--------------------
Obesity 2
ALT
0.4485412167428728
--------------------
Obesity 3
NIMAP
0.4642547816162228
--------------------


#### GPVAE

In [48]:
print("GPVAE - MAE")
print("************")
for i in range(len(subgroups)):
    print(subgroups[i]) 
    print("-------------")
    for j in range(len(variables)):
        print(variables[j], ":" ,testing_mae_gpvae_append_subgroups[i][j])

GPVAE - MAE
************
General
-------------
ALP : 0.4636806691897626
ALT : 0.43988335188854844
AST : 0.49487440139032896
Albumin : 0.5215603429773343
BUN : 0.4266804258106264
Bilirubin : 0.4362346111747271
Cholesterol : 0.45224918270847075
Creatinine : 0.4757477721136869
DiasABP : 0.44289760298360664
FiO2 : 0.4433338364096275
GCS : 0.4799413889558506
Glucose : 0.4696212244440866
HCO3 : 0.44450931999694504
HCT : 0.4501893885953067
HR : 0.4400535016898482
K : 0.4529856488282846
Lactate : 0.4461148095715683
MAP : 0.46808219341677
MechVent : 0.4524354149233376
Mg : 0.4669271249641094
NIDiasABP : 0.4272968366403261
NIMAP : 0.4424829097324757
NISysABP : 0.4687402557776164
Na : 0.4649150142299674
PaCO2 : 0.4297125605908459
PaO2 : 0.4616128061306596
Platelets : 0.4373574331464247
RespRate : 0.45190520545070256
SaO2 : 0.46773353160229914
SysABP : 0.44711432869783224
Temp : 0.44845590548679254
TroponinI : 0.43925062163985207
TroponinT : 0.4395172281370459
Urine : 0.47446754863420765
WBC : 0.4

In [49]:
df_gpvae_mae = pd.DataFrame(variables)

for i in range(len(subgroups)):
    df_gpvae_mae[subgroups[i]] = testing_mae_gpvae_append_subgroups[i]

In [50]:
df_gpvae_mae

,0,General,Female,Male,Undefined Gender,+65,-65,ICUType 1,ICUType 2,ICUType 3,ICUType 4,Undefined classification,Low Weight,Normal Weight,Overweight,Obesity 1,Obesity 2,Obesity 3
0,ALP,0.463681,0.437348,0.481185,0.188923,0.452031,0.500722,0.498392,0.398523,0.468410,0.453942,0.492038,0.401444,0.439143,0.440158,0.454013,0.354501,0.441009
1,ALT,0.439883,0.452353,0.424574,0.000000,0.491392,0.458706,0.486426,0.395146,0.451703,0.457088,0.503753,0.692737,0.422994,0.400253,0.492085,0.686335,0.415788
2,AST,0.494874,0.506453,0.450780,0.724645,0.482102,0.467843,0.510333,0.402633,0.515917,0.438254,0.490237,0.607884,0.429936,0.489471,0.417430,0.416185,0.333943
3,Albumin,0.521560,0.468480,0.511329,1.001637,0.443553,0.456627,0.458601,0.362322,0.501570,0.438250,0.441548,0.548004,0.448468,0.442848,0.500192,0.440872,0.377416
4,BUN,0.426680,0.446517,0.412046,0.150895,0.471250,0.431017,0.474564,0.397925,0.453885,0.427392,0.470006,0.563741,0.465482,0.483913,0.430060,0.423617,0.591936
5,Bilirubin,0.436235,0.479521,0.494864,0.734586,0.456773,0.467347,0.458350,0.400101,0.519027,0.488478,0.461902,0.496981,0.403788,0.446324,0.393508,0.463751,0.395469
6,Cholesterol,0.452249,0.428988,0.455132,0.145363,0.453071,0.474963,0.485967,0.381231,0.520121,0.471640,0.496510,0.305876,0.352159,0.400880,0.462925,0.515067,0.427294
7,Creatinine,0.475748,0.492619,0.491877,0.501357,0.436339,0.506338,0.455648,0.404820,0.508868,0.468376,0.486859,0.408171,0.432863,0.438505,0.453152,0.485521,0.450144
8,DiasABP,0.442898,0.462887,0.478513,0.051857,0.460529,0.553068,0.417038,0.388051,0.516924,0.470249,0.519733,0.484796,0.390566,0.396541,0.395753,0.448396,0.352941
9,FiO2,0.443334,0.478511,0.435290,0.350734,0.450360,0.482558,0.455084,0.370986,0.521568,0.461136,0.454381,0.324044,0.388751,0.413061,0.407168,0.443543,0.378275


Minimum MAE value in each subgroup

In [51]:
for i in range(len(subgroups)):
    value = df_gpvae_mae[subgroups[i]].min()
    print(subgroups[i])
    for j in range(len(variables)):
        if(df_gpvae_mae[subgroups[i]][j] == value):
            var = variables[j]
    print(var)
    print(value)        
    print("--------------------")
        

General
BUN
0.4266804258106264
--------------------
Female
HR
0.3975113856500141
--------------------
Male
PaCO2
0.40512053944486576
--------------------
Undefined Gender
Ph
0.0
--------------------
+65
Weight
0.40237378893918874
--------------------
-65
Temp
0.4213956061808997
--------------------
ICUType 1
NISysABP
0.3980735771511966
--------------------
ICUType 2
Temp
0.34595787233019126
--------------------
ICUType 3
TroponinT
0.44150717227931435
--------------------
ICUType 4
HCT
0.40540634000519604
--------------------
Undefined classification
TroponinT
0.43578454232106834
--------------------
Low Weight
Cholesterol
0.30587591187755
--------------------
Normal Weight
Cholesterol
0.3521591834844701
--------------------
Overweight
TroponinI
0.3669762146418675
--------------------
Obesity 1
Temp
0.3420051745969439
--------------------
Obesity 2
Lactate
0.31941491553707074
--------------------
Obesity 3
AST
0.3339430524408481
--------------------


Maximum MAE value in each subgroup

In [52]:
for i in range(len(subgroups)):
    value = df_gpvae_mae[subgroups[i]].max()
    print(subgroups[i])
    for j in range(len(variables)):
        if(df_gpvae_mae[subgroups[i]][j] == value):
            var = variables[j]
    print(var)
    print(value)        
    print("--------------------")
        

General
Albumin
0.5215603429773343
--------------------
Female
Urine
0.5218872656822563
--------------------
Male
Albumin
0.5113291034015329
--------------------
Undefined Gender
Albumin
1.0016368591369427
--------------------
+65
ALT
0.4913919110886271
--------------------
-65
DiasABP
0.5530684990194574
--------------------
ICUType 1
HCT
0.5732228618708284
--------------------
ICUType 2
GCS
0.4581674428405731
--------------------
ICUType 3
Urine
0.5654244430239622
--------------------
ICUType 4
NIMAP
0.525832966642237
--------------------
Undefined classification
GCS
0.523164650791944
--------------------
Low Weight
Ph
0.7472059997331083
--------------------
Normal Weight
Glucose
0.4964796364633969
--------------------
Overweight
AST
0.48947124273590503
--------------------
Obesity 1
HCO3
1.0711111868751826
--------------------
Obesity 2
Na
0.6898210398646814
--------------------
Obesity 3
NIMAP
0.69359485229345
--------------------


#### MRNN

In [53]:
print("MRNN - MAE")
print("************")
for i in range(len(subgroups)):
    print(subgroups[i]) 
    print("-------------")
    for j in range(len(variables)):
        print(variables[j], ":" ,testing_mae_mrnn_append_subgroups[i][j])

MRNN - MAE
************
General
-------------
ALP : 0.6891042237096028
ALT : 0.6697312359094596
AST : 0.7416927518816129
Albumin : 0.7463954870157028
BUN : 0.6355481879591666
Bilirubin : 0.6416969716355293
Cholesterol : 0.6727495200279108
Creatinine : 0.6716671861362805
DiasABP : 0.6808283782616357
FiO2 : 0.6227840389728461
GCS : 0.6692272659201319
Glucose : 0.6720891461117408
HCO3 : 0.6589864147836295
HCT : 0.6458595741032642
HR : 0.6621585497680288
K : 0.6896896911599226
Lactate : 0.6430019270508706
MAP : 0.6848244816182206
MechVent : 0.6811896916910888
Mg : 0.650124389963784
NIDiasABP : 0.6215348600145024
NIMAP : 0.6550581111501786
NISysABP : 0.6952255969268104
Na : 0.6844226542870258
PaCO2 : 0.6287560867465835
PaO2 : 0.6655651204171794
Platelets : 0.6457055708189681
RespRate : 0.6806955443585905
SaO2 : 0.6720348938943402
SysABP : 0.7056722589941172
Temp : 0.680304816992715
TroponinI : 0.6713200044286652
TroponinT : 0.6750343726015907
Urine : 0.7106147217278526
WBC : 0.6661850898761

In [54]:
df_mrnn_mae = pd.DataFrame(variables)

for i in range(len(subgroups)):
    df_mrnn_mae[subgroups[i]] = testing_mae_mrnn_append_subgroups[i]

In [55]:
df_mrnn_mae

,0,General,Female,Male,Undefined Gender,+65,-65,ICUType 1,ICUType 2,ICUType 3,ICUType 4,Undefined classification,Low Weight,Normal Weight,Overweight,Obesity 1,Obesity 2,Obesity 3
0,ALP,0.689104,0.667335,0.696357,0.507509,0.646565,0.687002,0.672778,0.620173,0.665549,0.670807,0.726986,0.701119,0.628677,0.624297,0.645220,0.575216,0.737773
1,ALT,0.669731,0.653103,0.654165,0.000000,0.725275,0.686937,0.749737,0.582993,0.685512,0.722545,0.730973,0.722608,0.655463,0.628625,0.628376,0.931739,0.845520
2,AST,0.741693,0.756573,0.673059,0.083308,0.705751,0.681755,0.695791,0.634816,0.798241,0.634615,0.712305,0.859331,0.675043,0.669676,0.591304,0.582306,0.681127
3,Albumin,0.746395,0.704606,0.727019,1.014342,0.673047,0.653678,0.674022,0.541006,0.717779,0.665241,0.684194,0.676810,0.665176,0.608718,0.683893,0.600455,0.590899
4,BUN,0.635548,0.699937,0.587745,0.475151,0.725550,0.627522,0.714661,0.647397,0.701698,0.619873,0.671836,0.772429,0.699477,0.665936,0.619232,0.630307,0.729258
5,Bilirubin,0.641697,0.700051,0.677235,0.408022,0.676139,0.655981,0.633770,0.604834,0.775192,0.679092,0.706135,0.533569,0.639532,0.638812,0.480346,0.669950,0.587823
6,Cholesterol,0.672750,0.669457,0.653943,0.269789,0.681913,0.652739,0.737699,0.605656,0.750578,0.646315,0.678274,0.646438,0.625275,0.602681,0.651379,0.839308,0.649382
7,Creatinine,0.671667,0.656672,0.728552,0.425493,0.649219,0.734455,0.650856,0.618545,0.738558,0.676408,0.698230,0.642483,0.664061,0.599628,0.655849,0.603157,0.854659
8,DiasABP,0.680828,0.680967,0.678367,0.561185,0.643803,0.815644,0.633256,0.588678,0.741413,0.635030,0.744099,0.770286,0.596444,0.554561,0.598039,0.468811,0.654385
9,FiO2,0.622784,0.697946,0.599732,0.278597,0.640812,0.669459,0.712953,0.560267,0.748908,0.682014,0.674755,0.651656,0.583494,0.645466,0.603720,0.625158,0.472036


Minimum MAE value in each subgroup

In [56]:
for i in range(len(subgroups)):
    value = df_mrnn_mae[subgroups[i]].min()
    print(subgroups[i])
    for j in range(len(variables)):
        if(df_mrnn_mae[subgroups[i]][j] == value):
            var = variables[j]
    print(var)
    print(value)        
    print("--------------------")
        

General
NIDiasABP
0.6215348600145024
--------------------
Female
TroponinT
0.62921742531125
--------------------
Male
PaCO2
0.5871804208287293
--------------------
Undefined Gender
Ph
0.0
--------------------
+65
Na
0.5893284527545951
--------------------
-65
NIDiasABP
0.6003298402934673
--------------------
ICUType 1
Temp
0.6057869251152296
--------------------
ICUType 2
MAP
0.5319993160382074
--------------------
ICUType 3
PaCO2
0.6184535399366906
--------------------
ICUType 4
NIDiasABP
0.593098916398076
--------------------
Undefined classification
HCT
0.6469828467080323
--------------------
Low Weight
Urine
0.4873191634699364
--------------------
Normal Weight
Mg
0.5709812876480205
--------------------
Overweight
MAP
0.526813176022993
--------------------
Obesity 1
Bilirubin
0.48034552174968403
--------------------
Obesity 2
WBC
0.446878399316957
--------------------
Obesity 3
FiO2
0.47203633593304234
--------------------


Maximum MAE value in each subgroup

In [57]:
for i in range(len(subgroups)):
    value = df_mrnn_mae[subgroups[i]].max()
    print(subgroups[i])
    for j in range(len(variables)):
        if(df_mrnn_mae[subgroups[i]][j] == value):
            var = variables[j]
    print(var)
    print(value)        
    print("--------------------")
        

General
Albumin
0.7463954870157028
--------------------
Female
AST
0.7565732521313827
--------------------
Male
Creatinine
0.7285521064107695
--------------------
Undefined Gender
MechVent
1.0643811764346358
--------------------
+65
BUN
0.7255498613054194
--------------------
-65
DiasABP
0.8156441578600511
--------------------
ICUType 1
HCT
0.8218244252064445
--------------------
ICUType 2
K
0.6633662789027489
--------------------
ICUType 3
TroponinI
0.842596466376899
--------------------
ICUType 4
SysABP
0.7555944034928384
--------------------
Undefined classification
SysABP
0.7544810939173036
--------------------
Low Weight
TroponinI
1.0675889203929847
--------------------
Normal Weight
Temp
0.7669488691837123
--------------------
Overweight
RespRate
0.7106134022293207
--------------------
Obesity 1
HCO3
1.2535198925354494
--------------------
Obesity 2
Na
1.0899931865042596
--------------------
Obesity 3
RespRate
0.961257497137432
--------------------
